# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_path = "../output_data/cities1.csv"

city_data = pd.read_csv(city_data_path)
 



### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=2.5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
city_data_1 = city_data.set_index("Unnamed: 0")


#Dataframe includes cities with "good weather", measured by low humidity (<80), no cloudiness, and moderate max temps (between 70 and 90 degrees)

city_data_nice = city_data_1.loc[(city_data_1["Humidity"] < 80) & (city_data_1["Cloudiness"] == 0) & (city_data_1["Max Temp"] < 90) & (city_data_1["Max Temp"] > 70)]


hotel_df = city_data_nice

hotel_df

#Create Column for Hotel Name
hotel_df["Hotel Name"] = ""
hotel_df.head()



<ipython-input-4-76588e3a7ecb>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Lat,Lng,Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
Unnamed: 0,,,,,,,,,,,
6,qandala,11.4720,49.8728,86.47,86.47,67,0,5.21,SO,1621967786,
11,arraial do cabo,-22.9661,-42.0278,73.26,75.16,56,0,4.61,BR,1621967788,
83,saint-pierre,-21.3393,55.4781,69.39,71.24,74,0,4.61,RE,1621967818,
104,quetta,30.1990,67.0097,72.84,72.84,19,0,1.95,PK,1621967589,
110,santa sofia,43.9445,11.9087,62.46,70.36,68,0,6.80,IT,1621967828,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}


#BH: Store Latitude and Longitude in locations
hotel_locations = hotel_df[["Lat", "Lng"]]

for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"


# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    hotel_info = requests.get(base_url, params)

    hotel_info = hotel_info.json()

#Get hotel names for each location
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_info["results"][0]["name"]
    except:
        print("Missing Hotel... Skipping")



hotel_df


Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping
Missing Hotel... Skipping


,City,Lat,Lng,Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
Unnamed: 0,,,,,,,,,,,
6,qandala,11.4720,49.8728,86.47,86.47,67,0,5.21,SO,1621967786,
11,arraial do cabo,-22.9661,-42.0278,73.26,75.16,56,0,4.61,BR,1621967788,
83,saint-pierre,-21.3393,55.4781,69.39,71.24,74,0,4.61,RE,1621967818,
104,quetta,30.1990,67.0097,72.84,72.84,19,0,1.95,PK,1621967589,
110,santa sofia,43.9445,11.9087,62.46,70.36,68,0,6.80,IT,1621967828,
151,sao filipe,14.8961,-24.4956,74.97,74.97,73,0,14.99,CV,1621967846,
170,erzin,36.9559,36.2000,74.01,75.92,64,0,2.75,TR,1621967855,
172,hurghada,27.2574,33.8129,86.04,86.04,28,0,3.44,EG,1621967855,
188,olhao,37.0286,-7.8411,73.29,73.29,53,0,8.90,PT,1621967827,


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig


# Display figure


Figure(layout=FigureLayout(height='420px'))